<a href="https://colab.research.google.com/github/daanishrasheed/Computer-Architecture/blob/master/notebooks/BuildWeekUnit2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 4*

---


# Model Interpretation 2

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your work.

- [ ] Continue to iterate on your project: data cleaning, exploratory visualization, feature engineering, modeling.
- [ ] Make a Shapley force plot to explain at least 1 individual prediction.
- [ ] Share at least 1 visualization (of any type) on Slack.

But, if you aren't ready to make a Shapley force plot with your own dataset today, that's okay. You can practice this objective with another dataset instead. You may choose any dataset you've worked with previously.

## Stretch Goals
- [ ] Make Shapley force plots to explain at least 4 individual predictions.
    - If your project is Binary Classification, you can do a True Positive, True Negative, False Positive, False Negative.
    - If your project is Regression, you can do a high prediction with low error, a low prediction with low error, a high prediction with high error, and a low prediction with high error.
- [ ] Use Shapley values to display verbal explanations of individual predictions.
- [ ] Use the SHAP library for other visualization types.

The [SHAP repo](https://github.com/slundberg/shap) has examples for many visualization types, including:

- Force Plot, individual predictions
- Force Plot, multiple predictions
- Dependence Plot
- Summary Plot
- Summary Plot, Bar
- Interaction Values
- Decision Plots

We just did the first type during the lesson. The [Kaggle microcourse](https://www.kaggle.com/dansbecker/advanced-uses-of-shap-values) shows two more. Experiment and see what you can learn!


## Links
- [Kaggle / Dan Becker: Machine Learning Explainability — SHAP Values](https://www.kaggle.com/learn/machine-learning-explainability)
- [Christoph Molnar: Interpretable Machine Learning — Shapley Values](https://christophm.github.io/interpretable-ml-book/shapley.html)
- [SHAP repo](https://github.com/slundberg/shap) & [docs](https://shap.readthedocs.io/en/latest/)

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install eli5
    !pip install pdpbox
    !pip install shap

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [3]:
from google.colab import files
uploaded = files.upload()

Saving plays.csv to plays.csv


In [4]:
import pandas as pd
df = pd.read_csv('plays.csv', engine='python')
print(df.shape)
df.head()

(14193, 27)


,gameId,playId,quarter,GameClock,down,yardsToGo,possessionTeam,yardlineSide,yardlineNumber,offenseFormation,personnel.offense,defendersInTheBox,numberOfPassRushers,personnel.defense,HomeScoreBeforePlay,VisitorScoreBeforePlay,HomeScoreAfterPlay,VisitorScoreAfterPlay,isPenalty,isSTPlay,SpecialTeamsPlayType,KickReturnYardage,PassLength,PassResult,YardsAfterCatch,PlayResult,playDescription
0,2017091004,37,1,15:00:00,0,0,DET,DET,35.0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,False,True,Kickoff,23.0,NaN,NaN,NaN,42,K.Redfern kicks 65 yards from DET 35 to ARZ 0....
1,2017091004,73,1,14:54:00,1,10,ARI,ARI,23.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"4 DL, 2 LB, 5 DB",0,0,0,0,False,False,NaN,NaN,-2.0,C,6.0,4,(14:54) (Shotgun) C.Palmer pass short right to...
2,2017091004,97,1,14:16:00,2,6,ARI,ARI,27.0,SINGLEBACK,"1 RB, 1 TE, 3 WR",6.0,NaN,"4 DL, 2 LB, 5 DB",0,0,0,0,False,False,NaN,NaN,NaN,NaN,NaN,2,(14:16) Da.Johnson up the middle to ARZ 29 for...
3,2017091004,118,1,13:34:00,3,4,ARI,ARI,29.0,SHOTGUN,"1 RB, 0 TE, 4 WR",5.0,4.0,"4 DL, 1 LB, 6 DB",0,0,0,0,False,False,NaN,NaN,7.0,C,7.0,15,(13:34) (Shotgun) C.Palmer pass short middle t...
4,2017091004,153,1,12:51:00,1,10,ARI,ARI,44.0,SHOTGUN,"1 RB, 2 TE, 2 WR",7.0,NaN,"4 DL, 3 LB, 4 DB",0,0,0,0,False,False,NaN,NaN,NaN,NaN,NaN,-2,(12:51) (Shotgun) Da.Johnson left end to ARZ 4...


In [5]:
print(len(df))

14193


In [6]:
df = df[~df['isSTPlay']]
print(len(df))

11558


In [7]:
df = df[~df['playDescription'].str.contains("kneels")]
print (len(df))

11428


In [8]:
df.tail(5)

,gameId,playId,quarter,GameClock,down,yardsToGo,possessionTeam,yardlineSide,yardlineNumber,offenseFormation,personnel.offense,defendersInTheBox,numberOfPassRushers,personnel.defense,HomeScoreBeforePlay,VisitorScoreBeforePlay,HomeScoreAfterPlay,VisitorScoreAfterPlay,isPenalty,isSTPlay,SpecialTeamsPlayType,KickReturnYardage,PassLength,PassResult,YardsAfterCatch,PlayResult,playDescription
14187,2017101501,4675,5,05:09:00,2,11,CHI,BAL,41.0,SINGLEBACK,"1 RB, 2 TE, 2 WR",7.0,7.0,"3 DL, 4 LB, 4 DB",24,24,24,24,False,False,NaN,NaN,1.0,I,NaN,0,(5:09) M.Trubisky pass incomplete short left t...
14188,2017101501,4697,5,04:26:00,3,11,CHI,BAL,41.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,6.0,"1 DL, 4 LB, 6 DB",24,24,24,24,False,False,NaN,NaN,18.0,C,0.0,18,(4:26) (Shotgun) M.Trubisky pass deep middle t...
14189,2017101501,4721,5,03:43:00,1,10,CHI,BAL,23.0,I_FORM,"2 RB, 2 TE, 1 WR",8.0,NaN,"3 DL, 4 LB, 4 DB",24,24,24,24,False,False,NaN,NaN,NaN,NaN,NaN,-2,(3:43) Jo.Howard right guard to BLT 25 for -2 ...
14190,2017101501,4742,5,03:00:00,2,12,CHI,BAL,25.0,I_FORM,"2 RB, 2 TE, 1 WR",9.0,NaN,"3 DL, 4 LB, 4 DB",24,24,24,24,False,False,NaN,NaN,NaN,NaN,NaN,1,(3:00) Jo.Howard left guard to BLT 24 for 1 ya...
14191,2017101501,4763,5,02:16:00,3,11,CHI,BAL,24.0,I_FORM,"2 RB, 2 TE, 1 WR",9.0,NaN,"3 DL, 4 LB, 4 DB",24,24,24,24,False,False,NaN,NaN,NaN,NaN,NaN,2,(2:16) Jo.Howard left tackle to BLT 22 for 2 y...


In [9]:
df = df.drop(df[df.quarter==5].index)

In [10]:
df.tail()

,gameId,playId,quarter,GameClock,down,yardsToGo,possessionTeam,yardlineSide,yardlineNumber,offenseFormation,personnel.offense,defendersInTheBox,numberOfPassRushers,personnel.defense,HomeScoreBeforePlay,VisitorScoreBeforePlay,HomeScoreAfterPlay,VisitorScoreAfterPlay,isPenalty,isSTPlay,SpecialTeamsPlayType,KickReturnYardage,PassLength,PassResult,YardsAfterCatch,PlayResult,playDescription
14168,2017101501,4129,4,01:37:00,0,0,BAL,CHI,2.0,I_FORM,"1 RB, 2 TE, 2 WR",7.0,NaN,"2 DL, 4 LB, 5 DB",22,24,24,24,False,False,NaN,NaN,NaN,NaN,NaN,0,TWO-POINT CONVERSION ATTEMPT. J.Flacco pass to...
14170,2017101501,4204,4,01:31:00,1,20,CHI,CHI,15.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,6.0,"1 DL, 5 LB, 5 DB",24,24,24,24,False,False,NaN,NaN,4.0,C,0.0,4,(1:31) (Shotgun) M.Trubisky FUMBLES (Aborted) ...
14171,2017101501,4228,4,01:09:00,2,16,CHI,CHI,19.0,SHOTGUN,"1 RB, 1 TE, 3 WR",5.0,4.0,"1 DL, 5 LB, 5 DB",24,24,24,24,False,False,NaN,NaN,NaN,S,NaN,-4,"(1:09) (No Huddle, Shotgun) M.Trubisky sacked ..."
14172,2017101501,4247,4,00:31:00,3,20,CHI,CHI,15.0,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,NaN,"1 DL, 5 LB, 5 DB",24,24,24,24,False,False,NaN,NaN,NaN,NaN,NaN,8,(:31) Jo.Howard right guard pushed ob at CHI 2...
14174,2017101501,4309,4,00:12:00,1,10,BAL,BAL,44.0,EMPTY,"0 RB, 2 TE, 3 WR",2.0,3.0,"1 DL, 4 LB, 6 DB",24,24,24,24,False,False,NaN,NaN,4.0,C,12.0,16,(:12) (Shotgun) J.Flacco pass short right to M...


In [11]:
def translate_game_clock(row):
    raw_game_clock = row['GameClock']
    quarter = row['quarter']
    minutes, seconds_raw = raw_game_clock.partition(':')[::2]

    seconds = seconds_raw.partition(':')[0]

    total_seconds_left_in_quarter = int(seconds) + (int(minutes) * 60)

    if quarter == 3 or quarter == 1:
        return total_seconds_left_in_quarter + 900
    elif quarter == 4 or quarter == 2:
        return total_seconds_left_in_quarter

if 'GameClock' in list (df.columns):
    df['secondsLeftInHalf'] = df.apply(translate_game_clock, axis=1)

if 'quarter' in list(df.columns):
    df['half'] = df['quarter'].map(lambda q: 2 if q > 2 else 1)

In [13]:
def yards_to_endzone(row):
    if row['possessionTeam'] == row['yardlineSide']:
        return 100 - row['yardlineNumber']
    else :
        return row['yardlineNumber']

df['yardsToEndzone'] = df.apply(yards_to_endzone, axis = 1)

In [14]:
df.head()

,gameId,playId,quarter,GameClock,down,yardsToGo,possessionTeam,yardlineSide,yardlineNumber,offenseFormation,personnel.offense,defendersInTheBox,numberOfPassRushers,personnel.defense,HomeScoreBeforePlay,VisitorScoreBeforePlay,HomeScoreAfterPlay,VisitorScoreAfterPlay,isPenalty,isSTPlay,SpecialTeamsPlayType,KickReturnYardage,PassLength,PassResult,YardsAfterCatch,PlayResult,playDescription,secondsLeftInHalf,half,yardsToEndzone
1,2017091004,73,1,14:54:00,1,10,ARI,ARI,23.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,"4 DL, 2 LB, 5 DB",0,0,0,0,False,False,NaN,NaN,-2.0,C,6.0,4,(14:54) (Shotgun) C.Palmer pass short right to...,1794,1,77.0
2,2017091004,97,1,14:16:00,2,6,ARI,ARI,27.0,SINGLEBACK,"1 RB, 1 TE, 3 WR",6.0,NaN,"4 DL, 2 LB, 5 DB",0,0,0,0,False,False,NaN,NaN,NaN,NaN,NaN,2,(14:16) Da.Johnson up the middle to ARZ 29 for...,1756,1,73.0
3,2017091004,118,1,13:34:00,3,4,ARI,ARI,29.0,SHOTGUN,"1 RB, 0 TE, 4 WR",5.0,4.0,"4 DL, 1 LB, 6 DB",0,0,0,0,False,False,NaN,NaN,7.0,C,7.0,15,(13:34) (Shotgun) C.Palmer pass short middle t...,1714,1,71.0
4,2017091004,153,1,12:51:00,1,10,ARI,ARI,44.0,SHOTGUN,"1 RB, 2 TE, 2 WR",7.0,NaN,"4 DL, 3 LB, 4 DB",0,0,0,0,False,False,NaN,NaN,NaN,NaN,NaN,-2,(12:51) (Shotgun) Da.Johnson left end to ARZ 4...,1671,1,56.0
5,2017091004,174,1,12:13:00,2,12,ARI,ARI,42.0,EMPTY,"1 RB, 2 TE, 2 WR",6.0,4.0,"4 DL, 3 LB, 4 DB",0,0,0,0,True,False,NaN,NaN,7.0,I,NaN,0,(12:13) C.Palmer pass incomplete short right t...,1633,1,58.0


In [18]:
df['personnel.offense'].value_counts()

1 RB, 1 TE, 3 WR                   6622
1 RB, 2 TE, 2 WR                   2208
2 RB, 1 TE, 2 WR                    888
1 RB, 3 TE, 1 WR                    507
2 RB, 2 TE, 1 WR                    245
1 RB, 0 TE, 4 WR                    128
2 RB, 0 TE, 3 WR                    119
0 RB, 1 TE, 4 WR                    107
6 OL, 1 RB, 2 TE, 1 WR              101
6 OL, 1 RB, 1 TE, 2 WR               95
6 OL, 2 RB, 1 TE, 1 WR               42
1 RB, 2 TE, 1 WR,1 DL                41
6 OL, 2 RB, 2 TE, 0 WR               34
2 RB, 3 TE, 0 WR                     24
1 RB, 1 TE, 2 WR,1 DL                24
0 RB, 2 TE, 3 WR                     24
6 OL, 1 RB, 3 TE, 0 WR               19
6 OL, 2 RB, 0 TE, 2 WR               19
6 OL, 1 RB, 0 TE, 3 WR               17
7 OL, 1 RB, 0 TE, 2 WR               14
3 RB, 1 TE, 1 WR                     12
1 RB, 3 TE, 0 WR,1 DL                 6
6 OL, 1 RB, 2 TE, 0 WR,1 DL           6
0 RB, 3 TE, 2 WR                      4
6 OL, 1 RB, 1 TE, 1 WR,1 DL           4


In [19]:
df.dtypes

gameId                      int64
playId                      int64
quarter                     int64
GameClock                  object
down                        int64
yardsToGo                   int64
possessionTeam             object
yardlineSide               object
yardlineNumber            float64
offenseFormation           object
personnel.offense          object
defendersInTheBox         float64
numberOfPassRushers       float64
personnel.defense          object
HomeScoreBeforePlay         int64
VisitorScoreBeforePlay      int64
HomeScoreAfterPlay          int64
VisitorScoreAfterPlay       int64
isPenalty                    bool
isSTPlay                     bool
SpecialTeamsPlayType       object
KickReturnYardage         float64
PassLength                float64
PassResult                 object
YardsAfterCatch           float64
PlayResult                  int64
playDescription            object
secondsLeftInHalf           int64
half                        int64
yardsToEndzone